# openWithHaloFinder.ipynb

This is an advanced tutorial using FIREreader, be warned!!

This notebook is best used on Stampede2, where the halo file and snapshot directories live. You can run this notebook, and host a Firefly server, on Stampede by following the instructions [here](https://github.com/ageller/Firefly/wiki/Hosting-Firefly-on-a-Cluster-Environment). 

In this notebook, we open the AHF halo files saved on Stampede and offset the snapshot coordinates, as well as convert them to physical units, to put the center of the main halo at our origin. This is optional, since you can always fly within Firefly to a point and set that as your origin, but more convenient (and exact!). 

We also calculate the radius from the halo center for each particle and update the filter keys so we can interactively filter by radius from within Firefly. 

#### Importantly, we do **not** call the `reader.run()` method, which would not give us the flexibility required to change our units/calculate the radii & temperature before we output to JSON. 



In [1]:
%load_ext autoreload
%autoreload 2

from FIREreader import FIREreader
from snapshot_utils import openSnapshot
from snapshot_utils import getTemperature
import numpy as np
import os
import h5py

In [2]:
## initialize reader object and choose simulation to run
reader = FIREreader()
reader.directory = r"\Users\shife\OneDrive\CIERA_REU\Snapshots\AlexRichingsCHIMES\output"
reader.snapnum = 500
## could read this from snapshot times
current_redshift=0
reader.writeStartup = False

## Open the AHF Halo file and extract the halo center and other parameters

In [3]:
def getTemperature(U_code,mu):
    """U_codes = res['u']
        mu = res['u']"""
    U_cgs = U_code*1e10
    gamma=5/3.
    kB=1.38e-16 #erg /K
    m_proton=1.67e-24 # g 
    mean_molecular_weight=mu*m_proton
    return mean_molecular_weight * (gamma-1) * U_cgs / kB # kelvin

## Setup the reader configuration and load data

In [4]:
## decide which part types to save to JSON
reader.returnParts = ['PartType0', 'PartType4']

## choose the names the particle types will get in the UI
reader.names = {'PartType0':'Gas', 
                  'PartType4':'Stars' }

In [5]:
#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()

## by what factor should you sub-sample the data (e.g. array[::decimate])
decimate = [10., 1.]

In [6]:
## load in the data from hdf5 files and put it into reader.partsDict
for i,p in enumerate(reader.returnParts):
    reader.decimate[p] = decimate[i]
    reader.returnKeys[p] = ['Coordinates', 'Density','Velocities']
    #Note: you should only try to filter on scalar values (like density).  
    #The magnitude of the Velocities are calculated in Firefly, and you will automatically be allowed to filter on it
    reader.addFilter[p] = [False, True, False]
    ## tell it to do the log of density when filtering
    reader.dolog[p] = [False, True, False]
    
    
    #NOTE: all dictionaries in the "options" reference the swapped names (i.e., reader.names) you define above.  
    #If you don't define reader.names, then you can use the default keys from the hdf5 files 
    #(but then you will see those hdf5 names in the Firefly GUI)
    pp = reader.names[p]
    ## set the initial size of the particles when the interface loads
    reader.options['sizeMult'][pp] = 0.5
    
## set the default colors when the interface loads
reader.options['color'] = {'Gas':  [1., 0.3, 0., 1.],  
                           'Stars':[0., 1., .85, 1]} 

## set the camera center to be at the origin (defaults to np.mean(Coordinates) otherwise)
##     later on we subtract out halo_center from coordinates but could instead make this halo_center
reader.options['center'] = np.array([0., 0., 0.])

## initialize filter flags and options
reader.defineFilterKeys()

## load in return keys from snapshot
filenames_opened = reader.populate_dict()

\Users\shife\OneDrive\CIERA_REU\Snapshots\AlexRichingsCHIMES\output\snapshot_500.hdf5
\Users\shife\OneDrive\CIERA_REU\Snapshots\AlexRichingsCHIMES\output\snapshot_500.hdf5
PartType4 has no Density


### Let's calculate the galactocentric radius, offset the coordinates by it while we're at it, then add the array to Firefly using the `addtodict` method

In [7]:
hubble=.702

## calculate the radius from the halo center
gas_radii = np.sum(reader.partsDict['PartType0']['Coordinates']**2,axis=1)**0.5
star_radii = np.sum(reader.partsDict['PartType4']['Coordinates']**2,axis=1)**0.5

## add new radius array to the dictionary using addtodict method
reader.addtodict(reader.partsDict,None,'PartType0','Radius',0,0,vals=gas_radii, filterFlag = True)
reader.addtodict(reader.partsDict,None,'PartType4','Radius',0,0,vals=star_radii, filterFlag = True)

### Let's convert the density to physical units

In [8]:
print(reader.partsDict.keys())

dict_keys(['PartType0', 'PartType4'])


In [9]:
# Code mass -> g , (code length)^-3 -> cm^-3 , g -> nHydrogen
DENSITYFACT=2e43*(3.086e21)**-3/(1.67e-24)
reader.partsDict['PartType0']['log10Density'] = reader.partsDict['PartType0']['log10Density']+np.log10(DENSITYFACT)

### Now let's load necessary supplemental data to calculate the temperature

In [10]:
## add temperature as a filtered quantity within the parts dict, but only for gas
all_gas_temperature = np.array([])
all_star_masses = np.array([])
for fname in reader.loadedHDF5Files:
    print(fname)
    with h5py.File(fname,'r') as handle:
        ## load necessary arrays to calculate temperature
        gas_group = handle['PartType0']
        InternalEnergy = np.array(gas_group['InternalEnergy'])
        ChimesMu = np.array(gas_group['ChimesMu'])
        
        ## calculate the temperature and append it to total array
        all_gas_temperature=np.append(
            all_gas_temperature,
            getTemperature(
                InternalEnergy, ChimesMu)
        )
        
        ## save stellar masses for vcom below
        all_star_masses=np.append(
            all_star_masses,
            np.array(handle['PartType4/Masses'])
        )
    
## track the Temperature array, do the log, and add it to be filtered
reader.addtodict(reader.partsDict,None,'PartType0','Temperature',sendlog = 1, sendmag = 0,vals = all_gas_temperature, filterFlag = True)

\Users\shife\OneDrive\CIERA_REU\Snapshots\AlexRichingsCHIMES\output\snapshot_500.hdf5


In [11]:
help(reader.addtodict)

Help on method addtodict in module FIREreader:

addtodict(d, snap, part, dkey, sendlog, sendmag, usekey=None, mfac=1.0, vals=None, filterFlag=False) method of FIREreader.FIREreader instance
    #adds an array to the dict for a given particle set and data file



### Let's add more filter keys

In [12]:
# open Snapshot and extract Stellar Formation Time
stars = openSnapshot(reader.directory,reader.snapnum,4,cosmological=0,keys_to_extract=['AgeGyr'],fnames=reader.loadedHDF5Files)

# add age in Myr to dict
reader.addtodict(reader.partsDict,None,'PartType4','Stellar Age',sendlog = 0, sendmag = 0,vals = stars['AgeGyr'] * 1000, filterFlag = True)

\Users\shife\OneDrive\CIERA_REU\Snapshots\AlexRichingsCHIMES\output\snapshot_500.hdf5


In [13]:
# open Snapshot and extract chimes abundance
abundance = openSnapshot(reader.directory,reader.snapnum,0,cosmological=0,keys_to_extract=['HIIAbundance'],fnames=reader.loadedHDF5Files)

# add chimes abundance to dict
reader.addtodict(reader.partsDict,None,'PartType0','HII Abundance',sendlog = 0, sendmag = 0,vals = abundance['HIIAbundance'], filterFlag = True)

### And now let's pre-filter certain parameters to our liking

In [14]:
# filter out stars older than 0.5 Gyr
stars_ind = np.where(stars['AgeGyr'] < 0.5)[0]

# adjust appropriate dictionary keys accordingly (i.e. density, velocity, radius, temperature, stellar age)
for x in list(reader.partsDict['PartType4'].keys()):
    print(x)
    if (x not in reader.nodecimate):
        print("filtered")
        reader.partsDict['PartType4'][x] = reader.partsDict['PartType4'][x][stars_ind]

Coordinates
filtered
Velocities
filtered
filterKeys
doSPHrad
Radius
filtered
Stellar Age
filtered


In [15]:
# filter out gas less dense than 1 particle/cm^3
gas_ind = np.where(reader.partsDict['PartType0']['log10Density'] > -1)[0]

# adjust appropriate dictionary keys accordingly (i.e. density, velocity, radius, temperature, stellar age)
for x in list(reader.partsDict['PartType0'].keys()):
    print(x)
    if (x not in reader.nodecimate):
        print("filtered")
        reader.partsDict['PartType0'][x] = reader.partsDict['PartType0'][x][gas_ind]

Coordinates
filtered
log10Density
filtered
Velocities
filtered
filterKeys
doSPHrad
Radius
filtered
log10Temperature
filtered


In [16]:
reader.dataDir = "AlexRichingsCHIMES_500"

In [17]:
## finish up, let's shuffle + decimate, add the GUI friendly names, and create our final JSON!
reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

decimating and shuffling ...
shuffling ... 
dataDir AlexRichingsCHIMES_500
writing JSON files ...
Gas
Stars
AlexRichingsCHIMES_500\FIREdataOptions.json
